# Import

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import sklearn
from sklearn.metrics.pairwise import sigmoid_kernel

import warnings
warnings.filterwarnings('ignore')

import json

from math import sqrt

# Read csv files

In [2]:
movies_metadata_df = pd.read_csv('movies_metadata.csv')
ratings_df = pd.read_csv('ratings_small.csv')
credits_df = pd.read_csv('credits.csv')
keywords_df = pd.read_csv('keywords.csv')
links_small_df = pd.read_csv('links_small.csv')

# Data Analysis

In [3]:
movies_metadata_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
movies_metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [5]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [7]:
credits_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [8]:
keywords_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [9]:
links_small_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


### Cleaning and Modifying movies_metadata_df csv file
* First make a copy of movies_metadata_df
* Rename the id column to movieId for later merge
* There are some data with NAN value witch we don't need them, so we drop them
* Convert Dtype of movieId coloumn to int for merging

In [10]:
movies_metadata_copy = movies_metadata_df.copy()

In [11]:
movies_metadata_copy.rename(columns=({ 'id': 'movieId'}), inplace=True)

In [12]:
movies_metadata_copy = movies_metadata_copy.dropna().reset_index(drop=True)

In [13]:
movies_metadata_copy['movieId'] = movies_metadata_copy['movieId'].astype(int)

In [14]:
movies_metadata_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693 entries, 0 to 692
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  693 non-null    object 
 1   belongs_to_collection  693 non-null    object 
 2   budget                 693 non-null    object 
 3   genres                 693 non-null    object 
 4   homepage               693 non-null    object 
 5   movieId                693 non-null    int32  
 6   imdb_id                693 non-null    object 
 7   original_language      693 non-null    object 
 8   original_title         693 non-null    object 
 9   overview               693 non-null    object 
 10  popularity             693 non-null    object 
 11  poster_path            693 non-null    object 
 12  production_companies   693 non-null    object 
 13  production_countries   693 non-null    object 
 14  release_date           693 non-null    object 
 15  revenu

### creating movie vs user matrix

In this movies_metadata_copy dataframe, as we will use movieId and title columns. We are creating new dataframe using this two columns. At the same time, as we will use userId, movieid and rating columns. We are creating new dataframe using this three columns from ratings_df.csv.

In [15]:
movie = movies_metadata_copy.loc[:,{"movieId","title"}]
rating = ratings_df.loc[:,{"userId","movieId","rating"}]

We are combining two dataframe (movies_metadata_df and ratings_df) and are creating movie vs user matrix.

In [16]:
data = pd.merge(movie,rating)
data = data.iloc[:1000000,:]
user_movie_table = data.pivot_table(index = ["title"],columns = ["userId"],values = "rating").fillna(0)
user_movie_table.head(10)

userId,2,3,4,5,6,7,8,9,10,12,...,659,661,662,664,665,666,667,668,670,671
title,,,,,,,,,,,,,,,,,,,,,
2046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
300,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Close Shave,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A View to a Kill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alien,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aliens vs Predator: Requiem,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
Alvin and the Chipmunks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
American Pie Presents: The Naked Mile,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Austin Powers in Goldmember,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Content Base Model

In [17]:
selected_column = ['movieId', 'title', 'genres']
mve = movies_metadata_copy.loc[:, selected_column]
mve

,movieId,title,genres
0,710,GoldenEye,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '..."
1,10634,Friday,"[{'id': 35, 'name': 'Comedy'}]"
2,755,From Dusk Till Dawn,"[{'id': 27, 'name': 'Horror'}, {'id': 28, 'nam..."
3,5894,Blue in the Face,"[{'id': 35, 'name': 'Comedy'}]"
4,9070,Mighty Morphin Power Rangers: The Movie,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
...,...,...,...
688,281338,War for the Planet of the Apes,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam..."
689,336890,Goon: Last of the Enforcers,"[{'id': 35, 'name': 'Comedy'}]"
690,10991,Pokémon: Spell of the Unknown,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
691,335988,Transformers: The Last Knight,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na..."


In [18]:
import ast

def make_genres(s):
    obj = ast.literal_eval(s)
    return '|'.join(map(lambda x: x['name'], obj))

In [19]:
mve['genres2'] = mve.apply(lambda x: make_genres(x['genres']), axis=1)
mve

,movieId,title,genres,genres2
0,710,GoldenEye,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",Adventure|Action|Thriller
1,10634,Friday,"[{'id': 35, 'name': 'Comedy'}]",Comedy
2,755,From Dusk Till Dawn,"[{'id': 27, 'name': 'Horror'}, {'id': 28, 'nam...",Horror|Action|Thriller|Crime
3,5894,Blue in the Face,"[{'id': 35, 'name': 'Comedy'}]",Comedy
4,9070,Mighty Morphin Power Rangers: The Movie,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Action|Adventure|Science Fiction|Family|Fantasy
...,...,...,...,...
688,281338,War for the Planet of the Apes,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",Drama|Science Fiction|War
689,336890,Goon: Last of the Enforcers,"[{'id': 35, 'name': 'Comedy'}]",Comedy
690,10991,Pokémon: Spell of the Unknown,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Adventure|Fantasy|Animation|Action|Family
691,335988,Transformers: The Last Knight,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",Action|Science Fiction|Thriller|Adventure


In [20]:
mve = mve.drop('genres', axis=1)
mve

,movieId,title,genres2
0,710,GoldenEye,Adventure|Action|Thriller
1,10634,Friday,Comedy
2,755,From Dusk Till Dawn,Horror|Action|Thriller|Crime
3,5894,Blue in the Face,Comedy
4,9070,Mighty Morphin Power Rangers: The Movie,Action|Adventure|Science Fiction|Family|Fantasy
...,...,...,...
688,281338,War for the Planet of the Apes,Drama|Science Fiction|War
689,336890,Goon: Last of the Enforcers,Comedy
690,10991,Pokémon: Spell of the Unknown,Adventure|Fantasy|Animation|Action|Family
691,335988,Transformers: The Last Knight,Action|Science Fiction|Thriller|Adventure


In [21]:
# mve['genres'] = mve['genres'].apply(json.loads)
# for index,i in zip(mve.index,mve['genres']):
#     list1 = []
#     for j in range(len(i)):
#         list1.append((i[j]['name'])) # the key 'name' contains the name of the genre
#     mve.loc[index,'genres'] = str(list1)

In [22]:
# generate a seprate column for each genre
movg = mve.copy()
movg['genres2'] = movg['genres2'].str.split('|')
for index,row in movg.iterrows():
    for g in row['genres2']:
        movg.at[index,g] = 1

movg = movg.fillna(0)
movg = movg.drop(['genres2'], axis=1)
movg

,movieId,title,Adventure,Action,Thriller,Comedy,Horror,Crime,Science Fiction,Family,...,Drama,Romance,Music,Mystery,Western,Documentary,History,War,TV Movie,Foreign
0,710,GoldenEye,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10634,Friday,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,755,From Dusk Till Dawn,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5894,Blue in the Face,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9070,Mighty Morphin Power Rangers: The Movie,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,281338,War for the Planet of the Apes,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
689,336890,Goon: Last of the Enforcers,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
690,10991,Pokémon: Spell of the Unknown,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
691,335988,Transformers: The Last Knight,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# user input movies
input = pd.DataFrame({'movieId':[1975,11,12,298,1271],'rating':[5.0,3.5,2.0,5.0,4.5]})
input = input.sort_values(by='movieId')
input

,movieId,rating
1,11,3.5
2,12,2.0
3,298,5.0
4,1271,4.5
0,1975,5.0


In [24]:
inputr = input['rating'].reset_index(drop=True)
inputr

0    3.5
1    2.0
2    5.0
3    4.5
4    5.0
Name: rating, dtype: float64

In [25]:
usermg = movg[movg['movieId'].isin(input['movieId'].tolist())]
usermg = usermg.drop('movieId', axis=1).drop('title', axis=1)
usermg = usermg.reset_index(drop=True)
usermg

,Adventure,Action,Thriller,Comedy,Horror,Crime,Science Fiction,Family,Fantasy,Animation,Drama,Romance,Music,Mystery,Western,Documentary,History,War,TV Movie,Foreign
0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
#calculate user profile
user_profile = inputr.transpose().dot(usermg)
user_profile

Adventure           8.0
Action              8.0
Thriller           10.0
Comedy              0.0
Horror              5.0
Crime               5.0
Science Fiction     3.5
Family              2.0
Fantasy             0.0
Animation           2.0
Drama               0.0
Romance             0.0
Music               0.0
Mystery             0.0
Western             0.0
Documentary         0.0
History             0.0
War                 4.5
TV Movie            0.0
Foreign             0.0
Name: rating, dtype: float64

In [27]:
#calculate score of each movie for our user
mov_score = (user_profile.transpose()*movg).sum(axis=1)/sum(user_profile)
mov_score = pd.DataFrame(data=mov_score)
mov_score = mov_score.rename(columns={0:'score'})
mov_score

,score
0,0.541667
1,0.000000
2,0.583333
3,0.000000
4,0.447917
...,...
688,0.166667
689,0.000000
690,0.416667
691,0.614583


In [28]:
#merge dataframe with original dataframe to bring titles
costum_movie2 = pd.merge(mov_score, mve, left_index=True, right_index=True).drop('genres2', axis=1)
costum_movie2 = costum_movie2.sort_values(by='score',ascending=False)

#score over 0.5 means 'good movie'
costum_movie2 = costum_movie2[costum_movie2['score']>0.5]
costum_movie2 = costum_movie2.fillna(0)
# costum_movie2

selected_column = ['score', 'movieId', 'title']
costum_movie2 = costum_movie2.loc[:, selected_column]
costum_movie2

,score,movieId,title
148,0.718750,36648,Blade: Trinity
258,0.645833,13387,Transporter 3
158,0.645833,11679,xXx: State of the Union
396,0.645833,58233,Johnny English Reborn
640,0.645833,291805,Now You See Me 2
236,0.645833,13054,Rise of the Footsoldier
255,0.645833,10764,Quantum of Solace
18,0.614583,10061,Escape from L.A.
556,0.614583,76341,Mad Max: Fury Road
551,0.614583,87101,Terminator Genisys


# Collaborative Model

In [29]:
movies_df = mve.copy()
movies_df = movies_df.drop('genres2', axis=1) # drop genres because in collaborative we dont need to know features of items
movies_df

,movieId,title
0,710,GoldenEye
1,10634,Friday
2,755,From Dusk Till Dawn
3,5894,Blue in the Face
4,9070,Mighty Morphin Power Rangers: The Movie
...,...,...
688,281338,War for the Planet of the Apes
689,336890,Goon: Last of the Enforcers
690,10991,Pokémon: Spell of the Unknown
691,335988,Transformers: The Last Knight


In [30]:
for i in movies_df['movieId']:
    print(i)

710
10634
755
5894
9070
2292
11
22586
8587
329
78
771
812
10020
954
532
77771
627
10061
238
11224
10948
11238
88
402
1103
531
1891
85
429
1892
348
218
105
756
89
199
578
607
1669
9714
714
948
11281
9552
165
196
3170
242
9325
12230
15969
87
36647
1091
9945
707
871
603
564
1893
36685
37135
2667
620
9638
658
657
646
699
709
253
660
18111
36643
863
25188
8374
2787
889
12476
955
668
691
698
682
12599
15158
16146
1588
681
708
843
10991
808
41050
10865
10992
1648
984
585
671
161
120
66946
425
1576
36586
9334
557
1894
4614
2501
608
818
7451
10575
27997
36669
121
2662
310
12
22
9257
277
11059
24
10009
10389
10147
860
122
17047
1487
10625
7131
667
700
36670
809
558
80
11247
2502
395
9806
163
36648
23336
2061
844
12540
3396
693
8968
561
187
11679
1895
272
1696
6519
8923
647
533
1656
8274
674
4550
10045
168705
1690
10982
834
10053
9682
950
9792
588
956
591
920
58
10947
9615
7484
1975
5890
14620
36557
9836
10032
1246
14277
1251
5393
8275
7516
1248
14761
1250
13185
1271
559
2269
11798
810
285
24200


In [31]:
ratings_df = ratings_df.drop('timestamp',axis=1) #drop time
ratings_df

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


In [32]:
inputMovies = pd.DataFrame({'movieId':[1975,11,12,298,1271],'rating':[5.0,3.5,2.0,5.0,4.5]})
inputMovies = inputMovies.sort_values(by='movieId')
inputMovies = pd.merge(inputMovies, movies_df) #add movieId for each movie
inputMovies

,movieId,rating,title
0,11,3.5,Star Wars
1,12,2.0,Finding Nemo
2,298,5.0,Ocean's Thirteen
3,1271,4.5,300
4,1975,5.0,The Grudge 2


In [33]:
#we need all ratings of other users to movies witch our user rate too
subsetratings = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
subsetratings

,userId,movieId,rating
116,3,1271,3.0
967,15,11,2.5
3113,19,11,3.0
3462,19,1271,3.0
4010,23,11,3.5
...,...,...,...
96156,641,11,4.0
97087,650,11,5.0
97454,654,11,4.5
98359,658,1271,5.0


In [34]:
#becuase of the subset above is to large we should seprate a subset of that witch have more important to us
# (users whose have more rating in common with our user)
# so we group them by userId and get top 100 of the list
ratingGroups = subsetratings.groupby('userId')

In [35]:
ratingGroups = sorted(ratingGroups, key=lambda x: len(x[1]),reverse=True)
ratingGroups = ratingGroups[:100]
ratingGroups

[(564,
         userId  movieId  rating
  82831     564       11     3.0
  82832     564       12     1.0
  83447     564     1271     5.0
  83754     564     1975     3.0),
 (294,
         userId  movieId  rating
  40157     294       11     3.0
  40158     294       12     4.0
  40355     294     1271     3.0),
 (452,
         userId  movieId  rating
  61437     452       11     4.0
  61725     452     1271     3.0
  61878     452     1975     0.5),
 (605,
         userId  movieId  rating
  90865     605       11     3.0
  91015     605     1271     4.0
  91091     605     1975     1.0),
 (19,
        userId  movieId  rating
  3113      19       11     3.0
  3462      19     1271     3.0),
 (30,
        userId  movieId  rating
  5052      30       11     4.0
  5293      30     1271     4.0),
 (57,
        userId  movieId  rating
  8782      57       11     5.0
  8932      57     1975     2.0),
 (95,
         userId  movieId  rating
  14513      95       11     4.0
  14603      95    

In [36]:
#this cell is a copy of original code and I didnt change it at all
#It calculate similarity between our user and other 100 users

#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in ratingGroups:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [37]:
pearsonCorrelationDict

{564: 0.7715167498104595,
 294: -0.9176629354822506,
 452: -0.9078412990032045,
 605: -0.49999999999999944,
 19: 0,
 30: 0,
 57: -1.0,
 95: -1.0,
 232: 1.0,
 235: 1.0,
 247: 1.0,
 292: 1.0,
 306: -1.0,
 311: -1.0,
 358: 1.0,
 396: -1.0,
 509: 0,
 518: 0,
 537: 1.0,
 547: 1.0,
 575: 1.0,
 580: -1.0,
 3: 0,
 15: 0,
 23: 0,
 58: 0,
 59: 0,
 63: 0,
 67: 0,
 68: 0,
 70: 0,
 86: 0,
 88: 0,
 91: 0,
 92: 0,
 93: 0,
 105: 0,
 110: 0,
 111: 0,
 117: 0,
 119: 0,
 121: 0,
 125: 0,
 130: 0,
 133: 0,
 156: 0,
 157: 0,
 161: 0,
 182: 0,
 199: 0,
 212: 0,
 213: 0,
 219: 0,
 220: 0,
 239: 0,
 242: 0,
 249: 0,
 252: 0,
 268: 0,
 280: 0,
 283: 0,
 284: 0,
 285: 0,
 312: 0,
 319: 0,
 342: 0,
 350: 0,
 353: 0,
 354: 0,
 361: 0,
 378: 0,
 380: 0,
 382: 0,
 385: 0,
 387: 0,
 388: 0,
 390: 0,
 391: 0,
 402: 0,
 416: 0,
 421: 0,
 422: 0,
 433: 0,
 447: 0,
 457: 0,
 461: 0,
 463: 0,
 468: 0,
 470: 0,
 472: 0,
 487: 0,
 491: 0,
 493: 0,
 496: 0,
 508: 0,
 511: 0,
 514: 0,
 516: 0,
 517: 0,
 529: 0}

In [38]:
#this is a dataframe of our dictionary witch sorted by similarity and return first 20 items of that because the original one is too large
tmp = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
tmp.columns = ['similarity']
tmp['userId'] = tmp.index
tmp = tmp.sort_values(by='similarity', ascending=False)
tmp = tmp.reset_index(drop=True)
tmp = tmp[:20]
tmp

,similarity,userId
0,1.000000,235
1,1.000000,232
2,1.000000,575
3,1.000000,358
4,1.000000,292
5,1.000000,247
6,1.000000,537
7,1.000000,547
8,0.771517,564
9,0.000000,378


In [39]:
sr = ratings_df[ratings_df['userId'].isin(tmp['userId'].tolist())] #sr is a subset of ratings witch are belongs to selected users
tmp = pd.merge(sr, tmp, on='userId', how='inner') #we gather rating and similarity for each record by merging 2 dataframe
tmp['w_rating'] = tmp['rating']*tmp['similarity'] #calculating weightedRating for each movie by similarity*rating
tmp = tmp.groupby('movieId')['similarity','w_rating'].sum() #calculating sum of rating and weightedRating for each movie
tmp['score'] = tmp['w_rating']/tmp['similarity'] #calculating score of each movie
tmp

,similarity,w_rating,score
movieId,,,
1,7.771517,30.586067,3.935662
2,3.771517,13.586067,3.602282
3,1.771517,3.314550,1.871024
5,3.771517,12.814550,3.397718
6,3.771517,10.271517,2.723445
...,...,...,...
157296,1.000000,3.000000,3.000000
160656,1.000000,3.500000,3.500000
160718,1.000000,4.000000,4.000000


In [40]:
costum_movies = tmp.sort_values(by='score', ascending=False)
costum_movies = costum_movies[costum_movies['score']>3] #ratings are between 0 and 5. so those are over 3 means favorit
costum_movies = pd.merge(costum_movies, movies_df, left_index=True, right_on='movieId').iloc[:,2:] #merge dataframes to get titles
costum_movies

,score,movieId,title
212,5.000000,1991,Death Proof
99,5.000000,984,Dirty Harry
15,5.000000,532,A Close Shave
377,5.000000,1771,Captain America: The First Avenger
123,5.000000,22,Pirates of the Caribbean: The Curse of the Bla...
...,...,...,...
134,3.180407,1487,Hellboy
42,3.125000,948,Halloween
102,3.071991,161,Ocean's Eleven
54,3.056904,1091,The Thing


### Merge Both results for make decision

In [41]:
from sklearn.preprocessing import MinMaxScaler
# find commen result from both recommender systems
recommended = pd.merge(costum_movies,costum_movie2,how='inner', on='movieId').drop('title_x',axis=1)

# normalize scores becuase of the different ranges
recommended['collaborative'] = MinMaxScaler().fit_transform(recommended.score_x.values.reshape(-1,1))
recommended['content_based'] = MinMaxScaler().fit_transform(recommended.score_y.values.reshape(-1,1))

# calculate final score 
recommended['final_score'] = (recommended['collaborative']+recommended['content_based'])/2

# clean dataframe and reset_index and sort by scores
recommended = recommended.drop(['score_x','score_y'],axis=1)
recommended = recommended.sort_values(by='final_score', ascending=False)
recommended = recommended.rename(columns={'title_y':'title'})
recommended = recommended.reset_index(drop=True)
# recommended

selected_column = ['movieId', 'title', 'collaborative', 'content_based', 'final_score']
recommended = recommended.loc[:, selected_column]
recommended

,movieId,title,collaborative,content_based,final_score
0,2080,X-Men Origins: Wolverine,0.650536,1.000000,0.825268
1,955,Mission: Impossible II,1.000000,0.000000,0.500000
2,348,Alien,0.782969,0.142857,0.462913
3,954,Mission: Impossible,0.863729,0.000000,0.431865
4,691,The Spy Who Loved Me,0.573613,0.000000,0.286806
5,700,Octopussy,0.573613,0.000000,0.286806
6,714,Tomorrow Never Dies,0.573613,0.000000,0.286806
7,956,Mission: Impossible III,0.573613,0.000000,0.286806
8,707,A View to a Kill,0.525778,0.000000,0.262889
9,440,Aliens vs Predator: Requiem,0.368219,0.142857,0.255538


# ensemble/hybrid Model

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Reader, Dataset, SVD
from surprise import dataset
from surprise.model_selection import KFold
from surprise.model_selection.validation import cross_validate

In [59]:
reader = Reader() # Used to parse a file containing ratings
df = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=5)
kf.split(df) # Split the data into folds

<generator object KFold.split at 0x00000220895D5DD0>

For solve the dummy "AttributeError: 'DataFrame' object has no attribute 'raw_ratings'" error
* Create the MyDataset class
* Make reader as MyReader
* Make another dataset by apply changes with MyDataset class as "data"

In [71]:
class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, df, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(df['userId'], df['movieId'], df['rating'])]
        self.reader=reader

In [72]:
myReader = Reader(line_format='user item rating', rating_scale=(1, 5))

In [73]:
data = MyDataset(ratings_df, myReader)

In [74]:
# Use Single Value Decomposition (SVD) for cross-validation and fitting
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

trainset = df.build_full_trainset()
svd.fit(trainset)

In [47]:
meta = movies_metadata_df.copy()

In [48]:
meta['overview'] = meta['overview'].fillna('')

In [49]:
meta = meta.drop([19730, 29503, 35587]) # Remove these ids to solve ValueError: "Unable to parse string..."

# Convert object to int64 for compatibility during merging
meta['id'] = pd.to_numeric(meta['id'])

In [50]:
# Convert float64 to int64
col=np.array(links_small_df['tmdbId'], np.int64)
links_small_df['tmdbId']=col

In [51]:
# Merge the dataframes on column "tmdbId"
meta.rename(columns={'id':'tmdbId'}, inplace=True)
meta = pd.merge(meta,links_small_df,on='tmdbId')
meta.drop(['imdb_id'], axis=1, inplace=True)
meta.head()

,adult,belongs_to_collection,budget,genres,homepage,tmdbId,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,movieId,imdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1,114709
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,2,113497
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,3,113228
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,4,114885
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,5,113041


In [52]:
# Remove stop words and use TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english')
# Construct TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(meta['overview'])
tfidf_matrix.shape

(9099, 29727)

In [53]:
# Compute cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# Get corresponding indices of the movies
indices = pd.Series(meta.index, index=meta['original_title']).drop_duplicates()

In [54]:
col=np.array(links_small_df['tmdbId'], np.int64)
links_small_df['tmdbId']=col

# Merge movies_metadata.csv and links_small.csv files
links_small_df = links_small_df.merge(meta[['title', 'tmdbId']], on='tmdbId').set_index('title')
links_index = links_small_df.set_index('tmdbId') # For label indexing

In [57]:
# Recommendation function
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = links_small_df.loc[title]['tmdbId'] # Get the corresponding tmdb id
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31] # Scores of the 30 most similar movies
    movie_indices = [i[0] for i in sim_scores]
    
    movies = meta.iloc[movie_indices][['title', 'vote_average', 'tmdbId']]
    movies['est'] = movies['tmdbId'].apply(lambda x: svd.predict(userId, links_index.loc[x]['movieId']).est) # Estimated prediction using svd
    movies = movies.sort_values('est', ascending=False) # Rank movies according to the predicted values
    movies.columns = ['Title', 'Vote Average', 'TMDb Id', 'Estimated Prediction']
    return movies.head(15) # Display top 15 similar movies

In [75]:
hybrid(1, 'The Conjuring')

,Title,Vote Average,TMDb Id,Estimated Prediction
7795,Midnight in Paris,7.4,59436,3.250130
5790,The Turning Point,6.0,61280,3.066810
5587,The Boston Strangler,6.7,26690,2.943006
9074,The Conjuring 2,7.0,259693,2.919780
3784,Things Behind the Sun,5.9,102933,2.842879
1965,The Texas Chain Saw Massacre,7.1,30497,2.829207
8726,The Borderlands,5.1,207774,2.776197
8262,Safe Haven,6.9,112949,2.758229
5859,Shining Through,6.2,31962,2.753017
3077,The Spiral Staircase,6.6,27452,2.730695


In [76]:
hybrid(30, 'The Conjuring')

,Title,Vote Average,TMDb Id,Estimated Prediction
7795,Midnight in Paris,7.4,59436,4.451797
7049,Magicians,6.2,10078,4.211907
1965,The Texas Chain Saw Massacre,7.1,30497,3.941857
3077,The Spiral Staircase,6.6,27452,3.871781
8389,Koch,5.0,138217,3.819174
8139,Sinister,6.8,82507,3.809379
5859,Shining Through,6.2,31962,3.766591
8993,Ashby,6.2,330112,3.701896
8262,Safe Haven,6.9,112949,3.679299
5364,Night of the Living Dead,6.7,19185,3.672381


In [77]:
hybrid(500, 'The Conjuring')

,Title,Vote Average,TMDb Id,Estimated Prediction
7795,Midnight in Paris,7.4,59436,3.656892
1965,The Texas Chain Saw Massacre,7.1,30497,3.263627
5859,Shining Through,6.2,31962,3.197075
3784,Things Behind the Sun,5.9,102933,3.169590
8976,Deathgasm,6.4,323373,3.060798
8993,Ashby,6.2,330112,2.991328
5790,The Turning Point,6.0,61280,2.984025
8139,Sinister,6.8,82507,2.964679
8726,The Borderlands,5.1,207774,2.955036
5587,The Boston Strangler,6.7,26690,2.945929
